In [11]:
# Step 1: Install required library
# !pip install scikit-learn pandas

In [12]:
# Step 2: Import required libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [20]:

# Step 3: Sample movie ratings dataset (can be replaced with your own)
data = {
    'User': ['A', 'A', 'A', 'B', 'B', 'C', 'C', 'D'],
    'Movie': ['Titanic', 'Avatar', 'Inception', 'Titanic', 'Inception', 'Avatar', 'Inception', 'Titanic'],
    'Rating': [5, 4, 5, 4, 3, 2, 5, 4]
}

df = pd.DataFrame(data)
print(df)

  User      Movie  Rating
0    A    Titanic       5
1    A     Avatar       4
2    A  Inception       5
3    B    Titanic       4
4    B  Inception       3
5    C     Avatar       2
6    C  Inception       5
7    D    Titanic       4


In [21]:

# Step 4: Pivot table - user-item matrix
pivot = df.pivot_table(index='User', columns='Movie', values='Rating').fillna(0)
print(pivot)

Movie  Avatar  Inception  Titanic
User                             
A         4.0        5.0      5.0
B         0.0        3.0      4.0
C         2.0        5.0      0.0
D         0.0        0.0      4.0


In [22]:
# Step 5: Compute cosine similarity between users
similarity = cosine_similarity(pivot)
similarity_df = pd.DataFrame(similarity, index=pivot.index, columns=pivot.index)

print("User Similarity Matrix:\n")
print(similarity_df)

User Similarity Matrix:

User         A         B         C         D
User                                        
A     1.000000  0.861640  0.754298  0.615457
B     0.861640  1.000000  0.557086  0.800000
C     0.754298  0.557086  1.000000  0.000000
D     0.615457  0.800000  0.000000  1.000000


In [23]:
# Step 6: Make a basic recommendation function
def recommend_movies(user):
    similar_users = similarity_df[user].sort_values(ascending=False)
    recommendations = pd.Series(dtype=float)

    for sim_user, score in similar_users.items():
        if sim_user == user or score < 0.1:
            continue
        user_ratings = pivot.loc[sim_user]
        recommendations = recommendations.add(user_ratings * score, fill_value=0)

    # Remove movies already rated by user
    rated = pivot.loc[user]
    recommendations = recommendations.drop(rated[rated > 0].index, errors='ignore')

    return recommendations.sort_values(ascending=False).head(3)

In [26]:
# Step 7: Example recommendation for User 'A'
print("\nTop Recommendations for User B:")
print(recommend_movies('B'))



Top Recommendations for User B:
Movie
Avatar    4.560734
dtype: float64
